# Top down Method
- 입력을 토대로 root부터 시작하여 인접한 노드중 방문하지 않은 노드들은 자식이므로, 그 정보를 이용해 자식 리스트를 만든다.
- 만들어진 자식 리스트를 이용해 다시 root부터 쿼리를 수행한다
- 입력 최적화가 됐음에도 상당히 느리다. (664ms)

In [ ]:
import io, os, sys
sys.setrecursionlimit(10**6)
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline
N, R, Q = map(int, input().split())
G = [[] for _ in range(N)]
P = [-1] * N
for _ in range(N-1):
  a, b = map(int, input().split())
  G[a-1].append(b-1)
  G[b-1].append(a-1)

child = [[] for _ in range(N)] #i 번째 노드의 자식들
def make_tree(cur, p) :
  for n in G[cur] : #연결된 노드중에서
    if n == p : continue #부모 노드가 아니면
    child[cur].append(n)
    P[n] = cur #내가 부모가 되며, 이 연결된 노드의 부모가 내가 된다.
    make_tree(n, cur)
make_tree(R-1, -1)

size = [1] * N
def q(cur) :
  for n in child[cur] :
    q(n)
    size[cur] += size[n]
q(R-1)

for _ in range(Q) :
  x = int(input())
  sys.stdout.write(f'{str(size[x-1])}\n')

# Bottom Up(후위 순회) Method
- stack을 이용해 후위 순회 리스트를 만들고, 후위 순회리스트의 특성을 이용해 서브트리 갯수의 값을 누적시킨다.
  - 후위 순위는 리프가 없는 자식 노드 -> 루트 노드 -> 형제 노드 순서대로 방문한다. 따라서 이때마다 방문하지 않은 자식 노드의 서브트리 갯수를 누적시키면 된다.
- 비효율적으로 처리되는 재귀가 없어 빠르다. (280ms)

In [ ]:
import io, os, sys
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

def sol() :
  N, R, Q = map(int, input().split())
  G = [[] for _ in range(N)]
  for _ in range(N-1):
    a, b = map(int, input().split())
    G[a-1].append(b-1)
    G[b-1].append(a-1)

  vis = [False] * N
  S = [R-1] #루트 노드 부터 시작
  PO = [] #후위 순위

  while S: #리프 뜯기
    cur = S[-1] #top, 바로 pop하지 않음에 유의
    vis[cur] = True #방문
    leaf = True
    for n in G[cur] :
      if not vis[n] : #방문하지 않은 노드 = 자식이 있는 노드 = 당장은 leaf가 아닌 노드
        S.append(n)
        leaf = False
    
    if leaf :
      PO.append(S.pop()) #리프 노드를 뜯어서 후위 순위 traversal에 넣는다.

  size = [1] * N #자기 자신을 포함한 서브트리의 크기
  for cur in PO: #이때 vis는 False가 이미 방문한 노드가 된다.
    for p in G[cur] : #연결된 노드중에서
      if vis[p] : #방문하지 않은 노드면 = 부모노드이면 = (후위순위이기 때문에)자식이 아니라는 뜻 
        size[p] += size[cur] #자식의 크기를 더한다.
    vis[cur] = False #방문

  for _ in range(Q) :
    x = int(input())
    sys.stdout.write(f'{str(size[x-1])}\n')

sol()

# 출력 조금더 최적화
- `Q`가 큰 경우 `sys.stdout.write`를 여러번 호출 하는 것보단 `join`을 이용해 한번에 출력하는 것이 더 빨랐다.
- list에서 append를 할 때 메모리 할당을 추가로 해야 할 때 느려지기 때문.
  - 또 하나 확인할 수 있는 점은 파이썬의 메모리 오버 할당에 대한 behavior이다.
  - `append`를 사용했을 때의 메모리 사용량은 143808이였고 정적 할당을 했을 때의 메모리 사용량은 143804였다. 
    - 어느시점에 얼만큼 초과할당을 했는진 모르지만, 어쨋든 초과할당을 한 것을 확인할 수 있다.

In [ ]:
answer = []
for i in range(Q) :
  x = int(input())
  answer.append(size[x-1])

sys.stdout.write('\n'.join(map(str, answer)))